In [ ]:
import usb.core
import usb.util
from PIL import Image, ImageOps
import numpy as np
from time import sleep
import os

In [ ]:
width = 1920
height = 1080
hz = 60
mode = 0x8100
#width = 800
#height = 600
#hz = 60
#mode = 0x4200

pixfmt = 0x2200

In [ ]:
def read(address):
    global dev

    data = [0] * 8
    data[0] = 0xb5
    data[1] = (address & 0xFF00) >> 8
    data[2] = address & 0xFF

    # bmRequestType: 0x21 - USB_DIR_OUT | USB_TYPE_CLASS | USB_RECIP_INTERFACE
    # bRequest:      0x09 - HID_REQ_SET_REPORT
    dev.ctrl_transfer(0x21, 0x09, 0x0300, 0, data)


    # bmRequestType: 0xa1 - USB_DIR_IN | USB_TYPE_CLASS | USB_RECIP_INTERFACE
    # bRequest:      0x01 - HID_REQ_GET_REPORT
    # we want to read 8 bytes
    data = dev.ctrl_transfer(0xa1, 0x01, 0x0300, 0, 8)

    # data now looks like this
    # idx |  0 | 1 | 2 | 3 | 4,5,6,7
    # val | b5 | H | L | x | follow-up bytes
    #            ^   ^   ^
    #           address  |
    # high and low byte  |
    #                    byte we wanted to read
    
    return data[3]

In [ ]:
def write6(address, six_bytes):
    global dev
    
    data = [0] * 2
    data[0] = 0xa6
    data[1] = address # on a write request, there is only one address byte...?
    data += six_bytes

    # bmRequestType: 0x21 - USB_DIR_OUT | USB_TYPE_CLASS | USB_RECIP_INTERFACE
    # bRequest:      0x09 - HID_REQ_SET_REPORT
    dev.ctrl_transfer(0x21, 0x09, 0x0300, 0, data)



In [ ]:
def power_on():
    write6(0x07, [1,2,0,0,0,0])

def power_off():
    write6(0x07, [0,0,0,0,0,0])
    # After powering off and on again a new call of set_resulution seems to be necessary

#detect if hdmi cable is connected
def detect():
    return read(0x32)


def set_resolution():
    global width, height, hz, mode, pixfmt
    
    write6(0x04, [0,0,0,0,0,0])
    read(0x30)
    read(0x33)
    read(0xc620)
    write6(0x03, [3,0,0,0,0,0])

    data = [
        (width & 0xFF00) >> 8,
        width & 0xFF,
        (height & 0xFF00) >> 8,
        height & 0xFF,
        (pixfmt & 0xFF00) >> 8,
        pixfmt & 0xFF
    ]
    write6(0x01, data)

    data = [
        (mode & 0xFF00) >> 8,
        mode & 0xFF,
        (width & 0xFF00) >> 8,
        width & 0xFF,
        (height & 0xFF00) >> 8,
        height & 0xFF
    ]
    write6(0x02, data)

    write6(0x04, [1,0,0,0,0,0])
    write6(0x05, [1,0,0,0,0,0])

In [ ]:
#color conversion. This is using numpy to speed things up.
#The whole process of loading an image from a file (i.e. call of load_image) takes about 2 seconds on a Raspberry Pi 3, 0.3 seconds on my computer.

#https://gist.github.com/Quasimondo/c3590226c924a06b276d606f4f189639
m = np.array([[ 0.29900, -0.16874,  0.50000],
             [0.58700, -0.33126, -0.41869],
             [ 0.11400, 0.50000, -0.08131]])

def rgb_to_yuv422(im):
    rgb = np.asarray(im)

    yuv422 = np.dot(rgb,m)
    yuv422[:,:,1:]+=128.0

    y = yuv422[:,:,0].reshape(width*height)
    u = yuv422[:,:,1].reshape(width*height)
    v = yuv422[:,:,2].reshape(width*height)


    #combine u and v.
    #this takes u for every element with even index, v for every element with odd index.
    uv = np.empty_like(u)
    uv[0::2] = u[0::2] 
    uv[1::2] = v[1::2]

    #"interlace" y and uv
    yuv = np.dstack((uv, y))
    yuv = yuv.reshape(-1)

    return yuv.astype(np.uint8).tolist()

# Opens an image specified by its file name and scales it to fit the screen. Written by ChatGPT
def scale_and_position_input_image(fname):
    global width, height
    original_image = Image.open(fname)

    original_image = ImageOps.exif_transpose(original_image)
    
    # Create a white canvas to paste the image on
    canvas_width = width
    canvas_height = height
    canvas = Image.new("RGB", (canvas_width, canvas_height), "black")

    # Calculate the dimensions and position of the image on the canvas
    img_width, img_height = original_image.size

    if img_width <= canvas_width and img_height <= canvas_height:
        # Center the image on the canvas
        position = ((canvas_width - img_width) // 2, (canvas_height - img_height) // 2)
        canvas.paste(original_image, position)
    else:
        # Scale the image to fit inside the canvas
        aspect_ratio = img_width / img_height
        if aspect_ratio > canvas_width / canvas_height:
            new_width = canvas_width
            new_height = int(canvas_width / aspect_ratio)
        else:
            new_height = canvas_height
            new_width = int(canvas_height * aspect_ratio)

        resized_image = original_image.resize((new_width, new_height))
        position = ((canvas_width - new_width) // 2, (canvas_height - new_height) // 2)
        canvas.paste(resized_image, position)

    return canvas

def load_image(fn):
    global width, height

    im = scale_and_position_input_image(fn)

    data = [
        0xff, 0, # header
        0, # start x / 16
        0, 0, # start y (High, Low)
        width // 16,
        (height & 0xFF00) >> 8, height & 0xFF
    ]

    # add image data in yuv422 format
    data += rgb_to_yuv422(im)

    # add footer
    data += [0xff, 0xc0, 0, 0, 0, 0, 0, 0]

    return bytes(data)

In [ ]:
# find our device
# use this udev rule on permission problems
# SUBSYSTEM=="usb", ATTRS{idVendor}=="534d", ATTRS{idProduct}=="6021", MODE="0666"

def get_device():
    dev = usb.core.find(idVendor=0x534d, idProduct=0x6021)

    if dev is None:
        raise ValueError('Device not found')

    # The kernel automatically binds usbhid to the control endpoint, we need to unbind it to do control transfers.
    
    if dev.is_kernel_driver_active(0): #control
        dev.detach_kernel_driver(0)
    if dev.is_kernel_driver_active(3): #bulk
        dev.detach_kernel_driver(3)

    return dev

def get_endpoint(dev):
    cfg = dev[0]
    intf = cfg[(3,0)]
    ep = intf[0]
    return ep

def wrapper_init(tries=2, do_reset=False):
    global dev
    
    if do_reset:
        dev.reset()
        
    dev = get_device()
    try:
        power_on()
        set_resolution()
    except:
        pass

    sleep(1)

    if tries == 2:        
        dev = get_device()
        power_on()
        set_resolution()

In [ ]:
wrapper_init()

In [ ]:
wrapper_init(1,True)

In [ ]:
ep = get_endpoint(dev)

In [ ]:
#example how to load two images (not included) and switch beteween them a few times
im1 = load_image("img.jpg")
im2 = load_image("img_2jpg")

for i in range(5):
    ep.write(im1)
    sleep(1)
    ep.write(im2)
    sleep(1)

In [ ]:
# a simple "slideshow" viewer that lets you display all images inside a directory (not included)
imagelist = os.listdir('images')
imagelist.sort()

this_image = None
next_image = load_image('images/'+imagelist[0])

for i in range(len(imagelist)):
    this_image = next_image
    
    ep.write(this_image)

    if not i+1 == len(imagelist):
        next_image = load_image('images/'+imagelist[i+1])
        
        input(imagelist[i]) #wait for user